# Amplifier classes
* Linear : A, B, C : 
* Switched mode : D, E, F
  * Class D : output capacitance of transistor is a loss element
  * **Class E** : output capacitance of transistor used in a tuned circuit, filter odd harmonics away (=loss)
  * Class F : improves on class E by adding tuned circuits for the odd harmonics as well.  In practice for QRP-use, the benefit is limited.

# Class-E topologies
[Scott Howard](https://raw.githubusercontent.com/KD9PDP/Minimalist-Class-E/master/PDF/minimalist_class_e.pdf) has compared Class E-amplifiers.  To have enough rejection of the harmonics, a Pi-match or T-match circuit is needed.  The Pi-match circuit is preferred because it gives more harmonic rejection and uses smaller inductor values.
We'll try both topologies to see what's best in our case.

# KD9PDP : Class E-amplifier with Pi-match notch filter
This topology is also used in the [(tr)uSDX receiver](./doc/truSDX_Main-RF_Boards_with_Notes_v1.0n.pdf).

![NM0S topology, image from KD9PDP paper](./doc/NM0S_pimatchnotch.png)

* I(L1) = 531mArms
* I(R1) = 349mArms
* η = 96%
* Harmonics:
  * 26.866MHz : 21.72dB
  * 53.766MHz : -32.05dB
  * 80.633MHz : -25.28dB

In [1]:
# Calculations for Pi-match with notch-filter
# according to KD9PDP

import math

# Design parameters
VCC = 12.0  # V
Pout = 5   # W
f = 27e6   # Hz
R1 = 50    # Ohm

Rl = 0.58 * math.pow(VCC, 2) / Pout  # Ohm, load seen by the MOSFET drain
Q = 3 + math.sqrt(Rl/5 - 1)

print("Q = %.2f" % Q)
print("Rl = %.2f Ohm" % Rl)

ω = 2*math.pi*f  # rad/s
L1 = 2.84 * Rl / ω          # H, minimum value for L1
C1 = 1 / (5.4466 * ω * Rl)  # F
Cf = 1 / (Q * ω * Rl)      # F
Ltot = (Q + 1.1525) * Rl / ω  # H
C2 = (Q-3) / (ω * Rl)         # F
L2 = 15 * Q / (4 * ω)       # H
C4 = 1 / (15*Q*ω)        # F
C3 = 3 / (50*ω)          # F

# LTSpice results
I_R1 = 0.349  # A
I_L1 = 0.531  # A
η = math.pow(I_R1,2)* R1 / (VCC*I_L1)
print("η = %.2f" % η)

print("L1 = %.2f uH" % (L1*1e6))
print("C1 = %.2f pF" % (C1*1e12))
print("Cf = %.2f pF" % (Cf*1e12))
print("Ltot = %.2f uH" % (Ltot*1e6))
print("C2 = %.2f pF" % (C2*1e12))
print("L2 = %.2f uH" % (L2*1e6))
print("C4 = %.2f pF" % (C4*1e12))
print("C3 = %.2f pF" % (C3*1e12))


Q = 4.53
Rl = 16.70 Ohm
η = 0.96
L1 = 0.28 uH
C1 = 64.79 pF
Cf = 77.90 pF
Ltot = 0.56 uH
C2 = 539.91 pF
L2 = 0.10 uH
C4 = 86.75 pF
C3 = 353.68 pF


# WA0ITP : Class-E amplifier with Pi-match notch filter
Calculation based on [WA0ITP's spreadsheet of the NM0S equations](http://www.wa0itp.com/classeampdesignrevb.xls)
![wa0itp pi-match notch design](./doc/WA0ITP_pimatchnotch.png)

The RdsOn and Coss have been set to zero to simulate a perfect switch.

* I(L1) = 431mArms
* I(R1) = 315mArms
* I(L2) = 791mArms
* I(L3) = 1.07Arms
* η = 96%
* Harmonics:
  * 26.87MHz : 20.86dB
  * 53.78MHz : -40.67dB
  * 80.63MHz : -49.90dB
  * 134.4MHz : -23.47dB

In [4]:
# LTSpice results
I_R1 = 0.315  # A
I_L1 = 0.431  # A
η = math.pow(I_R1,2)* R1 / (VCC*I_L1)
print("η = %.2f" % η)

η = 0.96


# T-match notch filter
![t-match notch design](./doc/tmatchnotch.png)

* I(L1) = 473mArms
* I(R1) = 328mArms
* η = 95%
* Harmonics:
  * 26.87MHz : 21.19dB
  * 53.77MHz : -41.19dB
  * 80.64MHz : -22.7dB

In [5]:
# Calculations for T-match with notch-filter according to KD9PDP
# Design parameters
VCC = 12.0  # V
Pout = 5   # W
f = 27e6   # Hz
Rant = 50    # Ohm
Q = 5

# Ltot and Cf are equal to the ones calculated for the Pi-match

Rl = 0.58 * math.pow(VCC, 2) / Pout  # Ohm, load seen by the MOSFET drain
Rv = Rl * (math.pow(Q, 2) + 1) # virtual load of the T-network
Q1 = math.sqrt(Rv/Rl - 1) # Q-factor of the T-network, seen from the MOSFET drain
Q2 = math.sqrt(Rv/Rant - 1) # Q-factor of the T-network, seen from the antenna

ω = 2*math.pi*f  # rad/s
L2 = Q1 * Rl / ω  # H
L3 = Q2 * Rant / ω  * 3/4 # H
C2 = 1 / (Rv * ω) * (Q1+Q2) # F
C3 = 1 / (math.pow(2*ω,2) * L3) # F

print('L2 = {:.2f}µH'.format(L2*1e6))
print('L3 = {:.2f}µH'.format(L3*1e6))
print('C2 = {:.2f}pF'.format(C2*1e12))
print('C3 = {:.2f}pF'.format(C3*1e12))

# LTSpice results
I_R1 = 0.328  # A
I_L1 = 0.473  # A
η = math.pow(I_R1,2)* R1 / (VCC*I_L1)
print("η = %.2f" % η)


L2 = 0.49µH
L3 = 0.61µH
C2 = 105.49pF
C3 = 14.17pF
η = 0.95



# Transistor limitations
* Transistor input capacitance Cin = Ciss = Cgs + Cgd
* Transistor output capacitance Cout = Coss = Cds + Cgd

# Output power calculations

$$Vout = VDD * \sqrt{D}$$

Where Vout = output voltage, VDD = supply voltage, D = duty cycle.

# Transistor selection
IRLML2060TRPBF
* 60V, 1.2A
* SOT23-3 package can be hand soldered.
* Ciss = 64pF
* Coss = 13pF
* Gate Charge (Qg) (Max) @ Vgs : 0.67 nC @ 4.5 V
* tr = 3.8ns, tf = 2.8ns
* Needs protection circuitry to handle over voltage when load is short-circuited.

IRLML0100TRPbF
* 100V, 1.6A
* SOT23-3
* tr = 2.1ns, tf = 3.6ns 
* Ciss = 290pF
* Coss = 27pF
* Qg = 2.5nC

# IRLML2060 versus IRLML0100
## IRLML2060
* Gate current = 62mA (5V)
* RF-choke current = 346mA (12V)
* 50ohm load current = 272mA
Input power = 4.46W
Output power = 3.7W
Efficiency = 83%

## IRLML0100
* Gate current = 137mA
* RF-choke current = 315mA
* 50ohm load current = 255mA
Input power = 4.47W
Output power = 3.25W
Efficiency = 73%

## Conclusion
The IRLML0100 can handle higher drain voltages, but simulations show that this part will break too when output is short circuited on an unprotected amplifier.  The IRLML0100 is less efficient, so there's little reason to choose this part over the IRLML2060.

In [6]:
import math
# Using Bill Slade calculations to calculate the required inductance and capacitance values for a 6.9 MHz amplifier

# Requirements
Vcc = 12  # [V]
Pload = 5  # [W]
f = 27e6  # [Hz]
Q = 5  # [] Tried with 10, but frequency band was too narrow
'''
Choice of Q is a trade-off between bandwidth and harmonic leakage.
------------------------------------------------------------------
Any value above 5-7 should suffice. Lower Qs will require “some tweaking” of
the excess inductance and shunt capacitor values as harmonic leakage becomes
significant. Low-Q permits operation over wider bandwidths, but at the cost
of more harmonic leakage (unless a higher-order output filter network is used).
'''
Rl = 50  # [Ω]


ω = 2*math.pi*f

# Calculate the required supply current
I0 = Pload / Vcc  # [A]
print("Required DC power supply current I0 = {:.2f} A".format(I0))

# Calculate the shunt capacitor (= sum of the parasitic Coss and the extra C)
Cp = I0 / (ω * math.pi * Vcc)  # [F]
print("Shunt capacitor Cp = {:.2f} pF".format(Cp * 1e12))

# Calculate the required load resistance seen by the transistor
R = 8*Vcc**2 / ((math.pi**2 + 4) * Pload)  # [Ω]
print("Required load resistance R = {:.2f} Ω".format(R))

# Calculate the excess series inductance Lext
Lext = 1.153 * R / ω  # [H]
print("Excess series inductance Lext = {:.2f} nH".format(Lext * 1e9))

# Calculate the series capacitor value
Cl = 1 / (Q * R * ω)  # [F]
print("Series capacitor Cl = {:.2f} pF".format(Cl * 1e12))

# Calculate the series inductance value
Lres = Q * R / ω  # [H]
print("Required series inductance Lres = {:.2f} nH".format(Lres * 1e9))

# Calculate the L match capacitor
# See also https://www.eeweb.com/tools/l-match/
Cm = 1 / (ω * Rl) * math.sqrt(Rl/R - 1)  # [F]
print("L match capacitor Cm = {:.2f} pF".format(Cm * 1e12))

# Calculate the L match inductance
L0 = R * Rl * Cm  # [H]
print("L match inductance L0 = {:.2f} nH".format(L0 * 1e9))

# Calculate the total inductance
L = Lext + Lres + L0  # [H]
print("Total inductance L = {:.2f} nH".format(L * 1e9))


Required DC power supply current I0 = 0.42 A
Shunt capacitor Cp = 65.15 pF
Required load resistance R = 16.61 Ω
Excess series inductance Lext = 112.90 nH
Series capacitor Cl = 70.97 pF
Required series inductance Lres = 489.60 nH
L match capacitor Cm = 167.14 pF
L match inductance L0 = 138.82 nH
Total inductance L = 741.33 nH


# Part selection

## RF-choke
1A, SRF>100MHz, L>3µH, package 1210
* BRL3225T4R7M, BRL3225T3R3M, CBC3225T3R3MR
Will put two or three of these in series.

# Protection

Needed in case of open load or shorted load.
## MOSFET-overvoltage protection
* TVS has too much capacitance.  Varistor and TVS have a too big difference between the breakdown voltage (start of clamping action) and the maximum clamping voltage.  
* Varistors : The maximum clamping voltage is about 60% to 100% higher than the breakdown voltage.
* TVS-diode : maximum clamping voltage is about 50% higher than the breakdown voltage.  Breakdown voltage is 20% above rated voltage.


# WA0ITP design
* 47dB attenuation of 2nd harmonic
* I(load) = 348mA, I(in) = 533mA -> 95% efficiency
* Small inductors
* 30 ohm output impedance
* small currents through coils
* Bandwidth?

# References
* [Search for the Minimalist QRP Class E Amplifier Filter Typology, KD9PDP](https://raw.githubusercontent.com/KD9PDP/Minimalist-Class-E/master/PDF/minimalist_class_e.pdf)
  * Must read
  * Good explanation of different Class-E topologies with calculations
* [Class E Amplifier Design Spreadsheet](http://wa0itp.com/class%20e%20design.html)
  * Implementation of David Cripe (NM0S) equations.
* [Notes on designing Class-E RF power ampliﬁers, George/Bill Slade](https://www.researchgate.net/publication/320623200)
  * Implementation of what KD9PDP describes as "2.1.1 “Power Boost” Class E".
  * Presented design requires high Q BPF to reduce harmonics enough.
* [Class E Amplifiers, Dan Tayloe, N7VE](http://norcalqrp.org/files/Class_E_Amplifiers.pdf)
  * protection circuits for open and shorted loads

## Other references (less interesting)
* [Modeling Class-E RF amplifiers](https://k6jca.blogspot.com/2011/01/modeling-class-ef-rf-power-amplifiers.html)
* [An Introduction to the uSDX transceiver kit](https://qrper.com/2020/09/an-introduction-to-the-usdx/)